In [1]:
import tensorflow as tf

In [2]:
import random

In [3]:
tf.set_random_seed(777)

In [4]:
import numpy as np

In [5]:
import sklearn.model_selection as sk
import pickle

## LOAD DATA

In [6]:
with open('noise_sounds_log.pickle','rb') as file:
    noise_log = pickle.load(file)

with open('snore_sounds_log.pickle','rb') as file:
    snore_log = pickle.load(file)

In [7]:
features = np.concatenate((noise_log, snore_log), axis=0)
#개수만큼 만들기
labels = np.array([[1,0] for _ in range(len(snore_log))]+[[0,1] for _ in range(len(noise_log))])

In [8]:
tr_features, ts_features, tr_labels, ts_labels = sk.train_test_split(features,labels,test_size=0.20, random_state = 77)

## DRAW GRAPH

In [9]:
learning_rate = 0.002 # 학습율(수치에 따라 기울기 내려가는 정도 조절 가능)

In [10]:
training_epochs = 30

In [11]:
batch_size = 100

In [12]:
# dropout (keep_prob) rate  0.7~0.5 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)

In [13]:
# input place holders
X_img = tf.placeholder(tf.float32, [None, 1025, 44, 1])

In [14]:
Y = tf.placeholder(tf.float32, [None, 2])

In [15]:
W1 = tf.Variable(tf.random_normal([2, 2, 1, 80], stddev=0.01),name='W1')

In [16]:
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 2, 2, 1], padding='SAME')

In [17]:
L1 = tf.nn.relu(L1)

In [18]:
# 여기서는 input tensor순서대로(1개 h,w,ch)
L1 = tf.nn.max_pool(L1, ksize=[1, 3, 2, 1], strides=[1, 1, 1, 1], padding='SAME')

In [19]:
# L2 ImgIn shape=(?, 60, 14, 80)
W2 = tf.Variable(tf.random_normal([2, 3, 80, 120], stddev=0.01),name="W2")

In [20]:
L2 = tf.nn.conv2d(L1, W2, strides=[1, 2, 2, 1], padding='SAME')

In [21]:
L2 = tf.nn.relu(L2)

In [22]:
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 1, 1, 1], padding='SAME')

In [23]:
L2_flat = tf.reshape(L2, [-1, 257*11*120]) # 스트라이드 주변 1행 1열 추가하여 1번 데이터 학습을 여러번 학습하기 위해서

In [24]:
# L4 FC 4x4x128 inputs -> 625 outputs
W4 = tf.get_variable("W4", shape=[257*11*120, 1000], initializer=tf.contrib.layers.xavier_initializer())

/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [25]:
b4 = tf.Variable(tf.random_normal([1000]),name="b4")

In [26]:
L4 = tf.nn.relu(tf.matmul(L2_flat, W4) + b4)

In [27]:
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

In [28]:
# L4 FC 4x4x128 inputs -> 625 outputs
W5 = tf.get_variable("W5", shape=[1000, 80], initializer=tf.contrib.layers.xavier_initializer())

In [29]:
b5 = tf.Variable(tf.random_normal([80]),name="b5")

In [30]:
L5 = tf.nn.relu(tf.matmul(L4, W5) + b5)

In [31]:
L5 = tf.nn.dropout(L5, keep_prob=keep_prob)

In [32]:
# L5 Final FC 625 inputs -> 10 outputs
W6 = tf.get_variable("W6", shape=[80, 2], initializer=tf.contrib.layers.xavier_initializer())

In [33]:
b6 = tf.Variable(tf.random_normal([2]),name="b6")

In [34]:
logits = tf.matmul(L5, W6) + b6

In [35]:
# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))

In [36]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

## SESSION

In [37]:
# initialize
sess = tf.Session()
saver = tf.train.Saver()


In [38]:
sess.run(tf.global_variables_initializer())

In [39]:
saver.restore(sess,'log_short-8')

INFO:tensorflow:Restoring parameters from log_short-8


In [40]:
# 계속 random한 값 리턴될수 있도록 하기(으 극혐)
def next_batch(num, data, labels):
    start =0
    randomly_selected_batch_idx = list(range(len(data)))
    random.shuffle(randomly_selected_batch_idx)
    
    for i in range(int(len(data)/num)):
        batch_idx = randomly_selected_batch_idx[start : start + num]
#         print(labels[batch_idx])s
        yield data[batch_idx],labels[batch_idx]
        start = start + num

In [41]:
import json
hi = {}

In [42]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [43]:
selected_list = random.sample(list(range(len(ts_features))),500)

In [44]:
a = ts_features[selected_list]
b = ts_labels[selected_list]

In [45]:
print('Epoch: 0001 cost = 2.939762530\n--accuracy 0001 cost = 0.895377100\nEpoch: 0002 cost = 0.256043833\nEpoch: 0003 cost = 0.250395121\nEpoch: 0004 cost = 0.154265839\nEpoch: 0005 cost = 0.131444436\nEpoch: 0006 cost = 0.108972554\n--accuracy 0006 cost = 0.964233577\nEpoch: 0007 cost = 0.135398123')
for epoch in range(7, training_epochs):
    avg_cost = 0
    total_batch = int(tr_features.shape[0] / batch_size)

    for batch_xs, batch_ys in next_batch(batch_size, tr_features, tr_labels):
        feed_dict = {X_img: batch_xs, Y: batch_ys, keep_prob: 0.5}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
    if epoch%5==0:
        accuracy_num = sess.run(accuracy, feed_dict={X_img:a, Y:b, keep_prob:1})
        print('--accuracy', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(accuracy_num))
        hi.update({epoch:avg_cost, 'accuracy':float(accuracy_num)})
    else:
        hi.update({epoch:avg_cost})
        
    with open('data.txt', 'w') as outfile:
        json.dump(hi, outfile)
        
    if (epoch+1)%10==0:
        saver.save(sess, "./log_short", global_step=epoch+1)
    
print('Learning Finished!')

Epoch: 0001 cost = 2.939762530
--accuracy 0001 cost = 0.895377100
Epoch: 0002 cost = 0.256043833
Epoch: 0003 cost = 0.250395121
Epoch: 0004 cost = 0.154265839
Epoch: 0005 cost = 0.131444436
Epoch: 0006 cost = 0.108972554
--accuracy 0006 cost = 0.964233577
Epoch: 0007 cost = 0.135398123
Epoch: 0008 cost = 0.088530318
Epoch: 0009 cost = 0.075170815
Epoch: 0010 cost = 0.073850613
Epoch: 0011 cost = 0.061645676
--accuracy 0011 cost = 0.967999995
Epoch: 0012 cost = 0.057414604
Epoch: 0013 cost = 0.054285760
Epoch: 0014 cost = 0.039042859
Epoch: 0015 cost = 0.089379947
Epoch: 0016 cost = 0.044905669
--accuracy 0016 cost = 0.966000021
Epoch: 0017 cost = 0.043235633
Epoch: 0018 cost = 0.048559445
Epoch: 0019 cost = 0.043084983
Epoch: 0020 cost = 0.035942665
Epoch: 0021 cost = 0.028541338
--accuracy 0021 cost = 0.967999995
Epoch: 0022 cost = 0.030718353
Epoch: 0023 cost = 0.027701679
Epoch: 0024 cost = 0.030910588
Epoch: 0025 cost = 0.031172793
Epoch: 0026 cost = 0.043373856
--accuracy 0026 cos

In [46]:
# SAVE LAST ONE
saver.save(sess, "./log_short", global_step=epoch+1)

'./log_short-30'

## 정확도

In [48]:
print('Accuracy:', sess.run(accuracy, feed_dict={
      X_img: ts_features, Y: ts_labels, keep_prob: 1}))

Accuracy: 0.96350366
